In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy

from statsmodels.sandbox.regression.gmm import GMM
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

In [4]:
data = pd.read_stata('chile.dta')
data = data.set_index(['id','year'])

In [5]:
ldata = data.copy()

ldata[['routput', 'totlab', 'renerg','realmats',
       'rcapstock']] = np.log(data[['routput','totlab', 'renerg','realmats','rcapstock']])

ldata = ldata.dropna()

do a non-parametric regression with $l_{it}$, $m_{it}$, $k_{it}$ on $y_{it}$ and save residual, and fitted value

given a guess of $\beta_1$ and $\beta_2$ compute an implied omega i.e. productive shock
get wit-1 (i.e. without estimating a function)
the residuals should be exogenous
these are the moment conditions

https://www.aeaweb.org/content/file?id=3015

In [6]:
d = ldata['routput'],ldata[['totlab']],ldata[['rcapstock','realmats','renerg']]

def np_resids(y,x):
    """residuals from lasso"""
    poly = PolynomialFeatures(degree=3)
    x_poly = poly.fit_transform(x)
    clf = Ridge(alpha=0.0)
    clf.fit(x_poly, y) 
    resid = y-clf.predict(x_poly)
    return resid

stage1_resids = np_resids(ldata['routput'],ldata[['totlab','rcapstock','realmats','renerg']])

In [7]:
def stage2obj(params, stage1_resids, dta):
    y,x = dta
    
    #compute value addeed to production by shock and all terms
    value = y - stage1_resids
    
    #compute implied value of endogenous shock
    shock = value - x.mul(params,axis=1).sum(axis=1)
    
    #####compute expected production trend############
    shocklag = shock.groupby('id').shift(1)
    both_shocks = pd.concat((shock, shocklag),axis=1).dropna()
    
    
    ###### save residuals ########
    shock2 = np_resids(both_shocks[[0]], both_shocks[[1]])[0]
    
    #####multiply by lags to get moment condition #########
    moments = x.mul(shock2,axis=0) 
    return moments.dropna()
    
d = ldata['routput'],ldata[['rcapstock','totlab']]

stage2obj(np.array([.5,.5]),stage1_resids, d)

rcapstock    totlab
id       year                     
18.0     1980   0.597190  0.185837
         1981  -3.933951 -1.053539
54.0     1980   1.287745  0.542679
         1981  -2.373567 -0.880470
         1982   1.824150  0.558133
         1983  -3.214530 -0.938018
         1984   0.488349  0.145848
         1985  -1.321634 -0.457237
         1986   2.073379  0.727611
         1987  -1.111578 -0.344963
126.0    1989  -7.853933 -4.135612
         1990  -1.560117 -0.991714
         1991  -0.300886 -0.154586
         1992  -0.184861 -0.094812
         1993  -2.129509 -1.056342
         1994   0.774054  0.389663
         1995  -1.508626 -0.759614
504.0    1980   2.333783  0.782109
         1981  -1.950131 -0.757377
         1982   1.437876  0.618618
         1983  -1.277956 -0.509569
513.0    1982  -1.564856 -1.027903
         1983 -19.522617 -6.266530
         1984 -10.182648 -2.963089
         1985  -6.072669 -1.776947
         1986  -6.730187 -1.419481
         1987  -1.257007 -0.402297
         1988 -10.146282 -2.705076
522.0    1980   0.869291  0.263702
         1981   0.502249  0.164641
...                  ...       ...
251235.0 1996   2.821853  1.235344
251388.0 1996  -5.484469 -1.840014
251433.0 1996   0.591183  0.198320
251478.0 1996  -2.278168 -0.950938
251676.0 1996   0.704591  0.291714
251712.0 1996   1.483968  0.446677
251748.0 1996   1.223042  0.464735
251829.0 1996  -3.949645 -1.857407
251838.0 1996  -1.515574 -0.782470
251847.0 1996   0.214943  0.147339
251874.0 1996   1.746455  0.715799
251883.0 1996  -3.593563 -1.732691
252081.0 1996   0.720452  0.256676
252207.0 1996   4.279367  1.303415
252315.0 1996  -3.616073 -1.705145
252324.0 1996   7.017380  3.355791
252513.0 1996  -0.889716 -0.357571
252666.0 1996  -3.899638 -1.350055
252882.0 1996   4.077684  1.490816
252918.0 1996  -3.877397 -1.328517
252963.0 1996 -10.429693 -4.246337
252972.0 1996   8.525009  3.216594
252981.0 1996 -11.517460 -3.493170
253215.0 1996   7.228365  2.913583
253224.0 1996   1.105509  0.496808
253269.0 1996   6.992450  2.885909
253341.0 1996  -7.227167 -2.561701
253431.0 1996  -0.590386 -0.189947
253593.0 1996   4.755022  1.348862
253629.0 1996 -11.969906 -7.268723

[18044 rows x 2 columns]

In [13]:
class ACF(GMM):
    
    def __init__(self, dta, stage1_resid, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        y,x = dta
        super(ACF, self).__init__(y,x,stage1_resid, x.shape[1],*args, **kwds)
        self.endog = y
        self.exog = x
        self.instr = stage1_resids
        
    def momcond(self, params):
        d = self.endog, self.exog
        return np.array( stage2obj(params, self.instr , d) )
        
        
d = ldata['routput'], ldata[['rcapstock','realmats']]
acfmodel = ACF(d, stage1_resids)
acfresult = acfmodel.fit(np.array([.1,.5]),optim_method='nm')
print acfresult.summary()

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 38
         Function evaluations: 74
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 21
         Function evaluations: 40
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 22
         Function evaluations: 41
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 22
         Function evaluations: 41
                                 ACF Results                                  
Dep. Variable:                routput   Hansen J:                    5.387e-06
Model:                            ACF   Prob (Hansen J):                   nan
Method:                           GMM                                         
Date:                Tue, 14 May 2019                                         
Time:                        15:05:26                